<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="Universidade Presbiteriana Mackenzie">
</head>

<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg" width=300, align="right">

<h1 align=left><font size = 8><b>Inteligência Artificial</b></font></h1>
<h1 align=left><font size = 6><b>Deep Learning</b></font></h1>

# Atividade: T4 - Atividade de aprofundamento

Nome: Bruno Rebocho de Toledo

Turma: 01B

Matrícula: 92316328

---
## Introdução

Este notebook tem como objetivo explorar as capacidades de classificação de redes neurais com TensorFlow e Keras.

---
## Dataset "Predict students' dropout and academic success"[1]

O conjunto de dados escolhido, criado a partir de uma instituição de ensino superior (adquirido de várias bases de dados independentes), descreve estudantes matriculados em diferentes graduações, como agronomia, design, educação, enfermagem, jornalismo, gestão, serviço social e tecnologias. O conjunto de dados inclui informações conhecidas no momento da matrícula do estudante (trajetória acadêmica, demografia e fatores socioeconômicos) e o desempenho acadêmico dos alunos ao final do primeiro e segundo semestres. Os dados são usados para construir modelos de classificação para prever a evasão e o sucesso acadêmico.

---
## O Problema

O problema é formulado como uma tarefa de classificação binária indicando a conclusão ou desistência do curso, na qual existe um forte desequilíbrio em favor de uma das classes originais.

---
## Explorando os Dados

Inicialmente, começamos explorando os dados ao importar as bibliotecas `numpy` e `pandas`. Utilizamos o `pandas` para ler o arquivo 'data.csv', especificando o ponto e vírgula como delimitador.

Após a leitura do arquivo, exibimos as primeiras cinco linhas do DataFrame `df` com a função `head()`. Esta visualização inicial nos fornece uma visão rápida de como os dados estão estruturados, incluindo nomes de colunas e tipos de dados, o que é crucial para o planejamento das etapas seguintes de análise.

Em seguida, imprimimos a dimensão do DataFrame com `df.shape`, revelando o número total de linhas e colunas. Esta informação é fundamental para entender o volume e a complexidade dos dados com os quais estamos trabalhando, orientando as decisões sobre o processamento e a análise subsequente.

Por vim, verificamos a presença de nulos ou valores ausentes, o que não é o caso deste nosso conjunto de dados.

In [ ]:
import numpy as np
import pandas as pd

!wget -O data.csv 'https://drive.google.com/uc?export=download&id=1hnHg7IN3qTPnIui2XrkT5hUd9XFH8Ecn'

df = pd.read_csv('data.csv', delimiter=';')

display(df.head())

print(df.shape)

In [ ]:
df.isnull().sum()

#### Avaliando a Distribuição da Coluna Alvo

Prosseguindo com a exploração dos dados, focamos na coluna 'Target', que desempenha um papel crucial na nossa análise. Para entender melhor a distribuição dos valores nesta coluna, utilizamos o método `value_counts()` do Pandas. Este método nos retorna rapidamente um panorama da frequência de cada valor único na coluna 'Target'.

Ao realizar esta operação, estamos buscando informações sobre a distribuição das classes ou categorias representadas na coluna 'Target'. Esta informação é vital, pois uma distribuição desigual das classes pode influenciar significativamente o desempenho do modelo.

Assim, ao imprimir o resultado de `value_counts()`, estabelecemos uma base para compreender como os dados estão estruturados em relação à variável que pretendemos prever.

Como esperado, há um forte desbalanceamento entre os valores.

In [ ]:
print(df['Target'].value_counts())

---
## Preparação dos Dados para Modelagem

Com o objetivo de adaptar nosso conjunto de dados para um problema de classificação binária, fizemos ajustes focando nas categorias 'Graduate' e 'Dropout'. Criamos a nova coluna 'Dropout' no DataFrame, codificando 'Graduate' como 0 e 'Dropout' como 1, que será nossa variável dependente. Eliminamos a categoria 'Enrolled', o que também contribui para reduzir o desbalanceamento inicial das classes.

Com a nova organização dos dados, removemos a coluna 'Target', pois ela já foi codificada em 'Dropout'. Em seguida, separamos o DataFrame em variáveis independentes `X` e a variável dependente `y`, que agora reflete com precisão o nosso problema binário de prever se um aluno irá se formar ou desistir.

In [ ]:
df = df[df['Target'].isin(['Graduate', 'Dropout'])]

df['Dropout'] = [0 if typ == 'Graduate' else 1 for typ in df['Target']]

df.drop('Target', axis=1, inplace=True)

X = df.drop('Dropout', axis=1)

y = df['Dropout']

---
## Separando os Dados em Treinamento e Teste

Agora avançamos para  a divisão do conjunto de dados em partes para treinamento e teste. Ao definirmos o parâmetro `test_size` como 0.3, estamos especificando que 30% do conjunto de dados será reservado para teste e 70% para treinar o modelo, conforme solicitado.

Após a aplicação da `train_test_split`, obtemos quatro conjuntos de dados: `X_train` e `y_train`, utilizados para treinamento, e `X_test` e `y_test`, que servirão para testar o modelo após o treinamento.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

---
## Normalizando os Dados

Iniciamos este processo utilizando o método `describe()` do DataFrame, que fornece um resumo estatístico, como média, desvio padrão, valores mínimos e máximos. Esta visão permite identificar a necessidade de normalização.

In [ ]:
df.describe()

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

---
## Definindo o Modelo de Rede Neural

Determinamos o número de características de entrada (`shape`) com base na forma do conjunto de dados de teste (`X_test`). Esse número guia a configuração da primeira camada da rede.

O modelo é construído como uma sequência de camadas, cada uma com a função de ativação `relu`, exceto a última camada que utiliza `sigmoid`. A primeira camada recebe `shape` neurônios, correspondendo ao número de características de entrada. A segunda camada tem metade do número de neurônios da primeira, seguindo uma estrutura que gradualmente condensa a informação. A última camada, com um único neurônio e ativação `sigmoid`, é adequada para uma tarefa de classificação binária, produzindo uma saída entre 0 e 1.

Após a definição do modelo, invocamos `model.summary()` para exibir um resumo da arquitetura da rede. Observamos que o modelo consiste em três camadas, com um total de 417 parâmetros.

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

shape = X_test.shape[1]

model = keras.Sequential([
    layers.Dense(shape, activation='relu', input_shape=[shape]),
    layers.Dense(shape / 2, activation='relu'),
    layers.Dense(1, activation='sigmoid'),
])

model.summary()

---
## Compilando o Modelo de Rede Neural

Escolhemos 'binary_crossentropy' como a função de perda e definimos 'binary_accuracy' como a métrica para avaliar o modelo.

In [ ]:
model.compile(
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)

---
## Treinamento do Modelo com Early Stopping

Utilizamos a técnica de early stopping para melhorar a eficiência e prevenir o overfitting.

Configuramos este callback para iniciar a avaliação da necessidade de parada após 10 épocas (`start_from_epoch=10`), com uma paciência de 10 épocas adicionais (`patience=10`). O parâmetro `restore_best_weights=True` garante que o modelo retorne aos pesos da época em que teve o melhor desempenho após a parada do treinamento.

Passamos `X_train` e `y_train` como os dados de treinamento, com um `batch_size` de 32. Utilizamos uma divisão de validação de 30% (`validation_split=0.3`) para avaliar o desempenho do modelo. O modelo é treinado para um máximo de 100 épocas (`epochs=100`), mas com a parada antecipada, podemos observar que o treinamento terminou antes deste limite.

Ao final do treinamento, o histórico é armazenado em `history`. Este histórico contém informações valiosas sobre a evolução da função de perda e das métricas de avaliação ao longo das épocas, permitindo a análise posterior do desempenho do modelo.

In [ ]:
from tensorflow import keras

early_stopping = keras.callbacks.EarlyStopping(
    start_from_epoch=10,
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
)

history = model.fit(
    X_train,
    y_train,
    batch_size=32,
    validation_split=0.3,
    epochs=100,
    callbacks=[early_stopping],
    verbose=1,
)

---
## Curva de Aprendizado e Avaliação do Desempenho do Modelo

Depois o treinamento, transformamos o histórico m um DataFrame para análise.

Os gráficos apresentam a evolução da perda e da acurácia ao longo das épocas. Observamos que a perda no conjunto de validação diminui de forma consistente ao longo do tempo, o que indica que o modelo está aprendendo efetivamente e melhorando a sua capacidade de generalização. Além disso, a acurácia binária no conjunto de validação aumenta e parece estabilizar, sugerindo que o modelo não está apenas memorizando os dados de treinamento, mas também se tornando eficaz em prever resultados em dados não vistos.

In [ ]:
history_df = pd.DataFrame(history.history)
display(history_df.head())
# Start the plot at epoch 0
history_df.loc[0:, ['loss', 'val_loss']].plot()
history_df.loc[0:, ['binary_accuracy', 'val_binary_accuracy']].plot()

print(("Best Validation Loss: {:0.4f}" +\
  "\nBest Validation Accuracy: {:0.4f}")\
  .format(history_df['val_loss'].min(),
          history_df['val_binary_accuracy'].max()))

---
## Avaliação Final do Modelo

Concluído o treinamento do modelo de rede neural, procedemos à sua avaliação usando o conjunto de teste (`X_test`). A predição do modelo é feita através do método `predict`, e um threshold de 0.5 é aplicado para determinar a classificação binária: valores iguais ou acima de 0.5 são considerados classe 1 (Dropout) e abaixo são classe 0 (Graduate).

A matriz de confusão revelada indica que o modelo teve um desempenho razoavelmente bom ao identificar as duas classes, com um número maior de verdadeiros positivos e verdadeiros negativos do que de falsos positivos e falsos negativos.

Em seguida, apresentamos o relatório de classificação, que oferece uma visão detalhada do desempenho do modelo, incluindo métricas como precisão, recall e pontuação f1 para cada classe. O relatório mostra que o modelo tem uma precisão e um recall balanceados para ambas as classes, resultando em uma boa pontuação f1.

Por último, calculamos a acurácia geral do modelo, que nos informa a proporção de previsões corretas sobre o total de previsões. Com uma acurácia de aproximadamente 88%, podemos concluir que o modelo é bastante eficaz.

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

y_pred = model.predict(X_test) >= 0.5

print('Matriz de Confusão: \n' , confusion_matrix(y_test, y_pred))

print(classification_report(y_test, y_pred))

print('Acuracidade: ' , accuracy_score(y_test, y_pred))

---
## Conclusões

Ao finalizar a análise e modelagem do conjunto de dados, concluímos que as redes neurais, com o uso do TensorFlow e Keras, demonstraram ser ferramentas eficazes para prever a evasão e o sucesso acadêmico de estudantes.

A aplicação de técnicas como normalização e early stopping contribuíram significativamente para a eficácia do modelo.

A precisão alcançada, em torno de 87%, sugere que o modelo pode ser uma ferramenta valiosa para identificar estudantes em risco, permitindo intervenções proativas para melhorar a retenção e o sucesso acadêmico.

---
## Referências

1. University of California, Irvine. Predict Student's Dropout and Academic Success. UCI Machine Learning Repository. Disponível em: https://archive.ics.uci.edu/dataset/697/predict+students+dropout+and+academic+success. Acesso em: 23 nov.2023.

2. BETTER-DATA-SCIENCE. TensorFlow Classification. 2023. Software. Disponível em: https://github.com/better-data-science/TensorFlow/blob/main/003_TensorFlow_Classification.ipynb. Acesso em: 22 nov.2023.